In [ ]:
class MinesweeperEnv:

    def __init__(self, height=9, width=9, num_mines=10):
        self.height = height
        self.width = width
        self.num_mines = num_mines
        self.reward = {'lose': -1, 'progress': 0.3, 'no_progress': -1, 'guess': 0.1, 'win': 1}

    def initialize_minefield(self):
        self.minefield = np.zeros((self.height, self.width), dtype=int)
        mines = set()
        while len(mines) < self.num_mines:
            x, y = random.randint(0, self.height - 1), random.randint(0, self.width - 1)
            if (x, y) not in mines:
                mines.add((x, y))
                self.minefield[x, y] = -2

        for (x, y) in mines:
            for i in range(max(0, x - 1), min(self.height, x + 2)):
                for j in range(max(0, y - 1), min(self.width, y + 2)):
                    if self.minefield[i, j] != -2:
                        self.minefield[i, j] += 1

    def reset(self):
        self.initialize_minefield()
        self.playerfield = np.full((self.height, self.width), -1, dtype=int)
        self.exploded = False
        self.done = False

        while True:
            sx, sy = random.randint(0, self.height - 1), random.randint(0, self.width - 1)
            if self.minefield[sx, sy] != -2:
                self.playerfield = self._uncover(sx, sy)
                if self.playerfield[sx, sy] == 0:
                    self.auto_reveal_blocks(sx, sy)
                break

        return self.normalize(self.playerfield)

    def step(self, action_idx):
        x, y = divmod(action_idx, self.width)
        reward = 0

        if self.playerfield[x, y] == -1:
            self.playerfield = self._uncover(x, y)
            if self.playerfield[x, y] == -2:
                reward = self.reward['lose']
                self.exploded = True
                self.done = True
            elif np.sum(self.playerfield == -1) == self.num_mines:
                reward = self.reward['win']
                self.exploded = False
                self.done = True
            else:
                near = [self.playerfield[i, j]
                        for i in range(x - 1, x + 2)
                        for j in range(y - 1, y + 2)
                        if (0 <= i < self.playerfield.shape[0] and 0 <= j < self.playerfield.shape[1]) and (i != x or j != y)]

                if any(value != -1 for value in near):
                    if self.playerfield[x, y] == 0:
                        self.auto_reveal_blocks(x, y)
                        reward = self.reward['progress']
                        self.exploded = False
                        self.done = False
                    else:
                        reward = self.reward['progress']
                        self.exploded = False
                        self.done = False
                else:
                    if self.playerfield[x, y] == 0:
                        self.auto_reveal_blocks(x, y)
                        reward = self.reward['guess']
                        self.exploded = False
                        self.done = False
                    else:
                        reward = self.reward['guess']
                        self.exploded = False
                        self.done = False
        else:
            reward = self.reward['no_progress']
            self.exploded = False
            self.done = False

        return self.normalize(self.playerfield), reward, self.exploded, self.done

    def _uncover(self, x, y):
        self.playerfield[x, y] = self.minefield[x, y]
        return self.playerfield

    def auto_reveal_blocks(self, x, y):
        queue = [(x, y)]
        while queue:
            cx, cy = queue.pop(0)
            for i in range(max(0, cx - 1), min(self.height, cx + 2)):
                for j in range(max(0, cy - 1), min(self.width, cy + 2)):
                    if self.playerfield[i, j] == -1:
                        self.playerfield = self._uncover(i, j)
                        if self.minefield[i, j] == 0:
                            queue.append((i, j))

    def normalize(self, playboard):
        return playboard / 8.0

    def render(self):
        render_str = ''
        for x in range(self.height):
            for y in range(self.width):
                if self.playerfield[x, y] == -1:
                    render_str += '. '  # Hidden
                elif self.playerfield[x, y] == -2:
                    render_str += 'M '  # Mine
                else:
                    render_str += f'{self.playerfield[x, y]} '
            render_str += '\n'
        print(render_str)